In [1]:
# Modify the file 00_setup.py to define input/output file paths on your system
# The information in 00_setup.py will be used across notebooks
from importlib.machinery import SourceFileLoader
setup = SourceFileLoader("setup", "./00_setup.py").load_module()

# 10: Data Imputation and Scaling for GNN
Convert data into a format suitable for GNN, preparing for several models
  * Modify data so it is suitable for a neural network
    * Impute missing features
    * Scale values
  * Create data frames for node and edge features
  * Get edge lists

I do not create graphs here, just do the preliminary data prep.  Not all features created here will be used in all models.  In addition, not all cases/edges would be used.  I will filter and modify the data to compare scenarios later.

*This script takes about 5 minutes on my MacBook Air*

In [2]:
import pandas as pd
import numpy as np

In [53]:
from pathlib import Path
import importlib, pickle
from sklearn import model_selection
from sklearn.preprocessing import LabelEncoder

In [4]:
# Imputer object for easy dataset conversion to GNN friendly format
from sba_gnn.sba_gnn import sg_imputer 
from sba_gnn.sba_gnn.sg_imputer import GNNImputer

## Input training data, descriptives, fit imputer

For a neural network, I need to handle missings and also scale features.  For missing values, I will do a simple median fill for all, but add missing indicators.  

To scale the data, I will do a quantile transform for features with > 5 levels.  This is to avoid scaling binary features.  Then I do a Min/Max scaling on all features, so they are in the [0,1] range

I define a class to do this in sg_imputer.  This class wraps several scikit-learn imputers/scalers so that I can easily fit the objects and then transform the data.  In this section, I show some descriptives to justify my imputation/scaling choices, and also fit the scaler

##### Import train data, show missing

In [5]:
sba_loans = pd.read_parquet(Path(setup.temp_path).joinpath('01_DATA_transformed.parquet'))

In [85]:
train_df = sba_loans[sba_loans['dset'] == 'train']

In [87]:
# Same information, more compact
pd.concat([pd.DataFrame([train_df[c].isna().sum()]) for c in setup.predictor_features],
         keys=setup.predictor_features).set_axis(['count_missing'], axis=1)

,,count_missing
NoEmp,0,0
CreateJob,0,0
LowDoc,0,3117
DisbursementGross,0,0
new_business,0,544
urban_flag,0,83339
franchise_flag,0,0


Most fields aren't missing too much.  However I must handle those that are.  I will use a median fill, especially since many of the fields with missingness seem to be binary

##### Explore scaling

In [88]:
# Level counts
feature_level_df = pd.concat([pd.DataFrame([train_df[c].value_counts().count()]) for c in setup.predictor_features],
         keys=setup.predictor_features)
feature_level_df

,,0
NoEmp,0,425
CreateJob,0,184
LowDoc,0,2
DisbursementGross,0,78795
new_business,0,2
urban_flag,0,2
franchise_flag,0,2


A number of features are binary.  I don't need to scale these.  The others must be scaled.  

What I will do is scale features with more than a threshold number of levels using a quantile scaler.  Then I'll use a MinMax scaler on everything to be safe (as a 2 level feature could have values other than 0,1).

##### Create imputer to do the median fill and scaling
This imputer fills missing values, adding missingness indicator features.  Then it quantile scales features with over 5 levels, and MinMax scales the rest.

In [306]:
importlib.reload(sg_imputer)
from sba_gnn.sba_gnn.sg_imputer import GNNImputer

In [307]:
imputer = GNNImputer(features = setup.predictor_features)                               

In [308]:
train_out = imputer.fit_transform(train_df)

In [309]:
train_out.describe()

,NoEmp,CreateJob,LowDoc,DisbursementGross,new_business,urban_flag,franchise_flag,missingindicator_LowDoc,missingindicator_new_business,missingindicator_urban_flag,NAICS
count,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000,425594.000000
mean,0.000952,-0.425126,-0.805820,-0.008115,-0.440904,0.702341,-0.898283,-0.985352,-0.997444,-0.608364,816.985373
std,0.575300,0.794153,0.592162,0.579310,0.897555,0.711842,0.439417,0.170532,0.071459,0.793659,342.255738
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000
25%,-0.435435,-1.000000,-1.000000,-0.510511,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,687.000000
50%,0.029029,-1.000000,-1.000000,-0.011011,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,897.000000
75%,0.505506,0.557558,-1.000000,0.492997,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1094.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1232.000000


In [311]:
with open(Path(setup.temp_path).joinpath('10_DATA_imputer.pkl'), 'wb') as fout:
    pickle.dump(imputer, fout)

In [276]:
with open(Path(setup.temp_path).joinpath('10_DATA_features.pkl'), 'wb') as fout:
    pickle.dump(imputer.features_out, fout)

In [312]:
naics_max_levels = imputer.get_naics_encoder_levels()
print(f'NAICS encoder max num: {naics_max_levels}')
with open(Path(setup.temp_path).joinpath('10_DATA_naics_max_levels.pkl'), 'wb') as fout:
    pickle.dump(naics_max_levels, fout)

NAICS encoder max num: 1232


## Transform Test, Validation Data

In [313]:
test_df = sba_loans[sba_loans['dset'] == 'test']
val_df = sba_loans[sba_loans['dset'] == 'val']

In [314]:
test_out = imputer.transform(test_df)

In [315]:
val_out = imputer.transform(val_df)

In [316]:
pd.concat([test_out[['DisbursementGross', 'NoEmp']], test_df[['DisbursementGross', 'NoEmp']]],
          axis=1).corr(method='spearman')

,DisbursementGross,NoEmp,DisbursementGross,NoEmp
DisbursementGross,1.000000,0.442105,1.000000,0.442105
NoEmp,0.442105,1.000000,0.442105,1.000000
DisbursementGross,1.000000,0.442105,1.000000,0.442105
NoEmp,0.442105,1.000000,0.442105,1.000000


In [317]:
val_out.describe()

,NoEmp,CreateJob,LowDoc,DisbursementGross,new_business,urban_flag,franchise_flag,missingindicator_LowDoc,missingindicator_new_business,missingindicator_urban_flag,NAICS
count,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000,126041.000000
mean,-0.000669,-0.421500,-0.802905,-0.008551,-0.442769,0.704160,-0.898049,-0.985402,-0.997937,-0.606557,814.446014
std,0.575287,0.795284,0.596109,0.579318,0.896639,0.710044,0.439897,0.170247,0.064198,0.795043,343.605139
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,-0.435435,-1.000000,-1.000000,-0.513514,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,680.000000
50%,0.029029,-1.000000,-1.000000,-0.011011,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,890.000000
75%,0.505506,0.557558,-1.000000,0.491491,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1093.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1232.000000


In [318]:
test_out.describe()

,NoEmp,CreateJob,LowDoc,DisbursementGross,new_business,urban_flag,franchise_flag,missingindicator_LowDoc,missingindicator_new_business,missingindicator_urban_flag,NAICS
count,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000,136446.000000
mean,0.014605,-0.424661,-0.802134,-0.010710,-0.455448,0.715477,-0.903889,-0.985122,-0.997420,-0.582223,616.290606
std,0.577628,0.794512,0.597146,0.578379,0.890266,0.698639,0.427769,0.171855,0.071784,0.813032,460.186502
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,0.000000
25%,-0.435435,-1.000000,-1.000000,-0.515055,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,31.250000
50%,0.029029,-1.000000,-1.000000,-0.011011,-1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,756.000000
75%,0.505506,0.557558,-1.000000,0.489428,1.000000,1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1057.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1232.000000


## Combine data
Combine transformed datasets, with key information.  Include the business ID, target status, NAICS features, as well as transformed features from above. Save this data for later use (especially the indices which indicate train/test statustrain_df

In [319]:
naics_features = ['LoanNr_ChkDgt', 'target', 'dset_naics_holdout', 'NAICS', 'NAICS_sector', 
                  'menc_NAICS', 'menc_NAICS_sector'] +  \
    [c for c in train_df.columns if c.startswith('NS__')]
print(naics_features)

['LoanNr_ChkDgt', 'target', 'dset_naics_holdout', 'NAICS', 'NAICS_sector', 'menc_NAICS', 'menc_NAICS_sector', 'NS___Accommodation and Food Services', 'NS___Construction', 'NS___Health Care and Social Assistance', 'NS___Manufacturing', 'NS___Other Services (except Public Administration)', 'NS___Professional, Scientific, and Technical Services', 'NS___Retail Trade', 'NS___Wholesale Trade', 'NS___infrequent_sklearn']


In [320]:
comb_naics = pd.concat([train_df[naics_features], 
                        test_df[naics_features], 
                        val_df[naics_features]], axis=0, keys=['train', 'test', 'val']) \
    .rename(columns={'NAICS':'NAICS_orig'})

In [321]:
comb_df = pd.concat([comb_naics,
                     pd.concat([train_out, test_out, val_out], 
                               axis=0, keys=['train', 'test', 'val'])],
                    axis=1) \
    .reset_index(level=0) \
    .rename(columns={'level_0':'dset'}, errors='ignore')
print(comb_df.shape)

(688081, 28)


In [322]:
print(comb_df.columns)

Index(['dset', 'LoanNr_ChkDgt', 'target', 'dset_naics_holdout', 'NAICS_orig',
       'NAICS_sector', 'menc_NAICS', 'menc_NAICS_sector',
       'NS___Accommodation and Food Services', 'NS___Construction',
       'NS___Health Care and Social Assistance', 'NS___Manufacturing',
       'NS___Other Services (except Public Administration)',
       'NS___Professional, Scientific, and Technical Services',
       'NS___Retail Trade', 'NS___Wholesale Trade', 'NS___infrequent_sklearn',
       'NoEmp', 'CreateJob', 'LowDoc', 'DisbursementGross', 'new_business',
       'urban_flag', 'franchise_flag', 'missingindicator_LowDoc',
       'missingindicator_new_business', 'missingindicator_urban_flag',
       'NAICS'],
      dtype='object')


In [323]:
comb_df.to_parquet(Path(setup.temp_path).joinpath('10_DATA_combined_scaled_all.parquet'))

In [324]:
comb_df[['target', 'urban_flag', 'menc_NAICS', 'menc_NAICS_sector', 'DisbursementGross']].corr()

,target,urban_flag,menc_NAICS,menc_NAICS_sector,DisbursementGross
target,1.000000,0.013001,0.232990,0.108193,-0.166215
urban_flag,0.013001,1.000000,-0.018373,-0.013307,-0.004150
menc_NAICS,0.232990,-0.018373,1.000000,0.451235,-0.257954
menc_NAICS_sector,0.108193,-0.013307,0.451235,1.000000,-0.130283
DisbursementGross,-0.166215,-0.004150,-0.257954,-0.130283,1.000000


## Create Edge Map

The following sections relate to graph neural networks.  I will create datasets for node features and relationships (edges)

I will have edges between a business and its NAICS, as well as edges to same-sector NAICS nodes.  This section creates general edge data, which may be filtered to create graphs

##### Create NAICS map to same-sector NAICS

In [325]:
sector_map = comb_df[['NAICS_orig', 'NAICS_sector']].drop_duplicates()

In [326]:
naics_all_sectors = sector_map.merge(sector_map.rename(columns={'NAICS_orig':'NAICS_sim'}),
                                     how='left', on ='NAICS_sector')
naics_all_sectors = naics_all_sectors[naics_all_sectors['NAICS_orig'] != naics_all_sectors['NAICS_sim']]

In [327]:
naics_all_sectors.sample(5)

,NAICS_orig,NAICS_sector,NAICS_sim
278114,334111,31-33,327331
272109,445299,44-45,441310
139121,332431,31-33,313111
272717,484220,48-49,485999
281184,314911,31-33,311520


##### Functions for creating index for NAICS

In [328]:
def get_naics_index(naics_seq):
    return 'n_' + naics_seq

##### Get edges from businesses to their exact NAICS

In [329]:
# Edges from businesses to their NAICS code
naics_info_df = comb_df[['LoanNr_ChkDgt', 'NAICS_orig', 'dset']].copy() \
    .rename(columns={'LoanNr_ChkDgt':'source'}) 
naics_info_df['target'] = get_naics_index(naics_info_df['NAICS_orig'])

edges_business_naics = naics_info_df[['source', 'target', 'dset']].copy()
edges_business_naics['type'] = 'loan_naics'

##### Get edges from buisinesses to related NAICS codes

In [330]:
naics_info_df.head(3)

,source,NAICS_orig,dset,target
index,,,,
0,1000014003,451120,train,n_451120
2,1000034009,621210,train,n_621210
7,1000094005,811118,train,n_811118


In [331]:
edges_businesses_naics_sim = naics_info_df.drop(columns='target', errors='ignore') \
    .merge(naics_all_sectors, how='inner', on='NAICS_orig')
edges_businesses_naics_sim['target'] = get_naics_index(edges_businesses_naics_sim['NAICS_sim'])
edges_businesses_naics_sim = edges_businesses_naics_sim[['source', 'target', 'dset']]
edges_businesses_naics_sim['type'] = 'loan_sector_naics'
print(f'Sector NAICS edges: {edges_businesses_naics_sim.shape}')

Sector NAICS edges: (67912366, 4)


##### Sample these - See 00_setup.py
The data is large so sampling could be considered here.  This is controlled in 00_setup.py.

In [332]:
if setup.naics_sector_sample:
    edges_businesses_naics_sim = edges_businesses_naics_sim.groupby('source') \
        .sample(setup.naics_sector_sample_n, replace=True, random_state=2342) \
        .drop_duplicates()
    print(f'Sector NAICS edges post sample: {edges_businesses_naics_sim.shape}')

Sector NAICS edges post sample: (11413065, 4)


In [333]:
edges_all = pd.concat([edges_business_naics, edges_businesses_naics_sim], axis=0) \
    .reset_index(drop=True)

In [334]:
edges_all['type'].value_counts()

type
loan_sector_naics    11413065
loan_naics             688081
Name: count, dtype: int64

In [335]:
print(edges_all[['source', 'target']].drop_duplicates().shape)
print(edges_all.shape)

(12101146, 2)
(12101146, 4)


In [336]:
edges_all.to_parquet(Path(setup.temp_path).joinpath('10_DATA_graph_edges.parquet'))

## Get Node Features
This section creates general node features data, which may be filtered to create graphs.  Not all features may be used, and not all nodes may be in all graphs.

In [337]:
# Business node features - these are the original features post scaling.
# Include the target also - will be used to test the "label trick" in some future scripts
features_business = comb_df[['LoanNr_ChkDgt', 'dset'] + imputer.features_out + ['target']] \
    .set_index('LoanNr_ChkDgt')
features_business.to_parquet(Path(setup.temp_path).joinpath('10_DATA_graph_node_features_loans.parquet'))

In [338]:
# NAICS features - One hots, also the mean encodings
features_naics = comb_df[['NAICS_orig', 'menc_NAICS', 'menc_NAICS_sector'] + [c for c in comb_df.columns if c.startswith('NS__')]] \
    .drop_duplicates()
features_naics['source'] = get_naics_index(features_naics['NAICS'])
features_naics.set_index('source', inplace=True) 
features_naics.drop(columns='NAICS_orig', inplace=True) 
print(features_naics.shape)
features_naics.to_parquet(Path(setup.temp_path).joinpath('10_DATA_graph_node_features_naics.parquet'))

KeyError: 'NAICS'

## Label Data
Split the training data set - used for some "label trick" tests later

In [ ]:
label_df = comb_df[['dset', 'dset_naics_holdout', 'LoanNr_ChkDgt', 'target']].set_index('LoanNr_ChkDgt')

In [ ]:
comb_df.columns

In [ ]:
train_labels = label_df[label_df['dset'] == 'train'].drop(columns=['dset'])
val_labels = label_df[label_df['dset'] == 'val'].drop(columns=['dset'])
print(f'graph labels shape {label_df.shape}')
print(f'train labels shape {train_labels.shape}')
print(f'validation labels shape {val_labels.shape}')

In [ ]:
# Split train cases for testing
train_fix, train_train = model_selection.train_test_split(
    train_labels.index, train_size=0.7, stratify=train_labels, random_state=23432
)

In [ ]:
print(f'fixed part of train set {len(train_fix)}')
print(f'train part of train set {len(train_train)}')

In [ ]:
label_df.train_fix = None
label_df.loc[train_fix, 'train_fix'] = 1
label_df['train_fix'].fillna(0, inplace=True)

In [ ]:
label_df['train_fix'].value_counts()

In [ ]:
len(set(list(train_fix)))

In [ ]:
label_df.to_parquet(Path(setup.temp_path).joinpath('10_DATA_label_info.parquet'))

In [ ]:
label_df.loc[train_fix].shape

In [ ]:
label_df.head()